In [ ]:
%matplotlib inline

import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib
from glob import glob
from PIL import Image, ImageFile

# matplotlib.use('nbAgg')
import matplotlib.pyplot as plt

In [ ]:
SMALL_SIZE = 10
MEDIUM_SIZE = 14
BIGGER_SIZE = 18
BIGGEST_SIZE = 22
plt.rc('font', size=BIGGEST_SIZE)         # controls default text sizes
plt.rc('axes', titlesize=BIGGEST_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=BIGGEST_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)   # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)   # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)   # legend fontsize
plt.rc('figure', titlesize=BIGGEST_SIZE)  # fontsize of the figure title

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_files
from sklearn.metrics import hamming_loss
from sklearn.metrics import average_precision_score
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.metrics import zero_one_loss
from sklearn.metrics import label_ranking_loss
from sklearn.metrics import coverage_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# Load Training, Validation, and Testing Data

In [ ]:
import cv2
from coco import CocoGenerator

subset_name = '2017'

coco_train_gen = CocoGenerator(subset_name=subset_name, split_name='train')
coco_train_sums_0 = np.sum(coco_train_gen.labels, axis=0, dtype=np.uint32)
coco_train_sums_1 = np.sum(coco_train_gen.labels, axis=1, dtype=np.uint8)
idx = np.random.randint(coco_train_gen.num_samples)
img = coco_train_gen.load_image(idx)[0]
plt.imshow(img)
plt.show()
print(coco_train_gen.labels.shape, idx, img.shape, img.min(), img.max())
print(coco_train_gen.label_names[np.where(coco_train_gen.labels[idx])[0]])

coco_valid_gen = CocoGenerator(subset_name=subset_name, split_name='valid')
coco_valid_sums_0 = np.sum(coco_valid_gen.labels, axis=0, dtype=np.uint32)
coco_valid_sums_1 = np.sum(coco_valid_gen.labels, axis=1, dtype=np.uint8)
idx = np.random.randint(coco_valid_gen.num_samples)
img = coco_valid_gen.load_image(idx)[0]
plt.imshow(img)
plt.show()
print(coco_valid_gen.labels.shape, idx, img.shape, img.min(), img.max())
print(coco_valid_gen.label_names[np.where(coco_valid_gen.labels[idx])[0]])

In [ ]:
max_train = np.max(coco_train_sums_1)
max_valid = np.max(coco_valid_sums_1)
max_tvt = np.max([max_train, max_valid])
n_images_with_k_tags_dict = {'train': [0]*int(max_tvt+1), 'valid': [0]*int(max_tvt+1)}

for s in coco_train_sums_1:
    n_images_with_k_tags_dict['train'][s] += 1
for s in coco_valid_sums_1:
    n_images_with_k_tags_dict['valid'][s] += 1

n_images_with_k_tags_df = pd.DataFrame(n_images_with_k_tags_dict)
n_images_with_k_tags_df.plot(kind='bar', figsize=[16, 3], secondary_y=('valid'))

In [ ]:
n_images_with_k_tags_df.style

In [ ]:
images_per_tag_dict = {'train': coco_train_sums_0, 'valid': coco_valid_sums_0}
images_per_tag_df = pd.DataFrame(images_per_tag_dict, index=coco_train_gen.label_names)
images_per_tag_df.plot(kind='bar', figsize=[16, 3], secondary_y=('valid'))

In [ ]:
images_per_tag_df.style

# Pretrain Top Model

In [ ]:
from transfer import KerasTransfer

kt = KerasTransfer(
    dataset='coco',
    subset='2017',
    base_pooling='avg',
    pp_method='zmuv',
    hidden_size_1=4096,
    hidden_size_2=4096,
    batch_norm=False,
    dropout=5,
    learn_rate_pretrain=(1, 1),
    batch_size_pretrain=32,
    resume_pretraining=True,
    epochs=500
)

In [ ]:
print(kt.top_model_weights_file)
kt.pretrain()

## Load Model

In [ ]:
kt.load_final_model()

In [ ]:
plt.rcParams['figure.dpi'] = 72

In [ ]:
csv_top_model_template = '../data/bottleneck_top_models/coco*.csv'
min_losses = []
for csv_file in sorted(glob(csv_top_model_template)):
    csv_basefile = csv_file.rsplit(os.sep, maxsplit=1)[1]
    csv_data = pd.read_csv(csv_file)
    min_idx = np.argmin(csv_data.val_loss)
    n_epochs = len(csv_data.val_loss)
    min_losses.append((csv_basefile, n_epochs, n_epochs - min_idx, csv_data.val_loss[min_idx]))

for label, n_ep, from_end, min_loss in sorted(min_losses, key=lambda x: x[3]):
    print("{:8.5f} {:3} {:2} {}".format(min_loss, n_ep, from_end, label))

In [ ]:
pset = ('coco', '2017', 'zmuv', 'avg', '1x2048bn', ['lr52', 'lr32', 'lr22', 'lr12', 'lr83', 'lr53', 'lr13'], 'b64')

colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(13, 9))

min_losses = []
in_prefix = True
prefix_list = []
suffix_list = []
for field in pset:
    if isinstance(field, str):
        if in_prefix:
            prefix_list.append(field)
        else:
            suffix_list.append(field)
    elif isinstance(field, list):
        labels = field
        in_prefix = False
prefix = '_'.join(prefix_list)
suffix = '_'.join(suffix_list)
pset_title = '_'.join([prefix, "????", suffix])
labelsA = [l + ' (train)' for l in labels]
labelsB = [l + ' (val)' for l in labels]
legend_labels = []
for k, label in enumerate(labels):

    csv_basefile = '_'.join([prefix, label, suffix])
    csv_file = "../data/bottleneck_top_models/" + csv_basefile + ".csv"
    csv_data = pd.read_csv(csv_file)
    csv_data.loss.plot(ax=axes, color=colors[k % len(colors)], style='--')
    csv_data.val_loss.plot(ax=axes, color=colors[k % len(colors)])
    legend_labels.append(labelsA[k])
    legend_labels.append(labelsB[k])
    
axes.set_title(pset_title)
axes.set_ylim((0.05, 0.16))
axes.set_xlabel(r'Epoch', {'fontsize': 20})
axes.set_ylabel(r'Loss', {'fontsize': 20})
axes.legend(legend_labels, loc='upper right', bbox_to_anchor=(0.98, 0.97))    

In [ ]:
pp_pl_c = [
    ("inet", "77", "k"),
    ("inet", "max", "r"),
    ("inet", "avg", "g"),
    ("zmuv", "77", "c"),
    ("zmuv", "max", "y"),
    ("zmuv", "avg", "b"),
]

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))
legend_labels = []
for pp, pl, c in pp_pl_c:
    csv_file = "../data/bottleneck_top_models/coco_2017_" + pp + "_" + pl + "_1x1024d5_lr11_b32.csv"
    csv_data = pd.read_csv(csv_file)
    axes.plot(csv_data.epoch, csv_data.val_loss, color=c)
    legend_labels.append(', '.join([pp, pl]))
    
axes.set_title("coco_2017_????_???_1x1024d5_lr11_b32")
axes.set_xlabel(r'Epoch')
axes.set_ylabel(r'Validation Loss')
# axes.set_ylim((0.065, 0.085))
axes.legend(legend_labels, ncol=2, loc='upper right', bbox_to_anchor=(0.99, 0.99))

In [ ]:
pl_sz_c = [
    ("max", "1024", "y"),
    ("max", "2048", "r"),
    ("max", "4096", "g"),
    ("avg", "1024", "b"),
    ("avg", "2048", "c"),
    ("avg", "4096", "m"),
]

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))
legend_labels = []
for pl, sz, c in pl_sz_c:
    csv_file = "../data/bottleneck_top_models/coco_2017_zmuv_" + pl + "_1x" + sz + "d5_lr11_b32.csv"
    csv_data = pd.read_csv(csv_file)
    axes.plot(csv_data.epoch, csv_data.val_loss, color=c)
    legend_labels.append(', '.join([pl, sz]))
    
axes.set_title("coco_2017_zmuv_???_1x????d5_lr11_b32")
axes.set_xlabel(r'Epoch')
axes.set_ylabel(r'Validation Loss')
# axes.set_ylim((0.065, 0.085))
axes.legend(legend_labels, ncol=2, loc='upper right', bbox_to_anchor=(0.99, 0.99))

In [ ]:
hyper_dict = {'nl': ['1', '2'], 'sz': ['1024', '2048', '4096'], 'lr': ['11', '52'], 'bs': ['32', '64', '128']}

epoch_time = {
    ("1", "1024", "32"): 17.5,
    ("1", "1024", "64"): 9.,
    ("1", "1024", "128"): 5.,
    ("1", "1024", "256"): 2.5,
    ("1", "2048", "32"): 18.,
    ("1", "2048", "64"): 9.,
    ("1", "2048", "128"): 5.,
    ("1", "2048", "256"): 2.5,
    ("1", "4096", "32"): 19.,
    ("1", "4096", "64"): 10.,
    ("1", "4096", "128"): 5.,
    ("1", "4096", "256"): 3.,
    ("2", "1024", "32"): 19.,
    ("2", "1024", "64"): 10.,
    ("2", "1024", "128"): 6.,
    ("2", "1024", "256"): 3.2,
    ("2", "2048", "32"): 22.,
    ("2", "2048", "64"): 12.,
    ("2", "2048", "128"): 8.,
    ("2", "2048", "256"): 4.2,
    ("2", "4096", "32"): 28.,
    ("2", "4096", "64"): 18.,
    ("2", "4096", "128"): 11.,
    ("2", "4096", "256"): 6.5,
    ("2", "4096", "512"): 4.,
}

RUST = '#d62728'
PINK = '#e377c2'
GREY = '#7f7f7f'
BROWN = '#8c564b'
PURPLE = '#9467bd'
ORANGE = '#ff7f0e'
SEA_BLUE = '#1f77b4'
LIME_GREEN = '#32cd32'
LIGHT_SALMON = '#FFA07A'

special_color = {
    "coco_2017_zmuv_avg_1x1096d5_lr11_b128": LIME_GREEN,
    "coco_2017_zmuv_avg_1x2048d5_lr11_b128": 'g',
    "coco_2017_zmuv_avg_1x4096d5_lr11_b128": 'y',
    "coco_2017_zmuv_avg_1x4096d5_lr52_b128": RUST,
    "coco_2017_zmuv_avg_2x1024d5_lr11_b128": ORANGE,
    "coco_2017_zmuv_avg_2x1024d5_lr52_b128": BROWN,
    "coco_2017_zmuv_avg_2x2048d5_lr11_b128": 'r',
    "coco_2017_zmuv_avg_2x2048d5_lr52_b128": PINK,
    "coco_2017_zmuv_avg_2x4096d5_lr11_b32": PURPLE,
    "coco_2017_zmuv_avg_2x4096d5_lr11_b64": 'm',
    "coco_2017_zmuv_avg_2x4096d5_lr11_b128": 'c',
    "coco_2017_zmuv_avg_2x4096d5_lr52_b32": GREY,
    "coco_2017_zmuv_avg_2x4096d5_lr52_b64": LIGHT_SALMON,
    "coco_2017_zmuv_avg_2x4096d5_lr52_b128": 'b',
    "coco_2017_zmuv_avg_2x4096d5_lr11_b256": 'k',
}

def plot_compare(const_dict, var_arr, in_seconds=False, ylim=(0.065, 0.085), xlim=None, ncol=2):
    
    def get_var():
        if a_is_free:
            v = a
            a_is_free = False
        elif b_is_free:
            v = b
            b_is_free = False
        else:
            raise ValueError
        return v

    top_models_dir = "../data/bottleneck_top_models/"
    coco_prefix = "coco_2017_zmuv_avg_"
    
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))
    legend_labels = []
    for a, b, c in var_arr:
        a_is_free = True
        b_is_free = True
        csv_basename = coco_prefix
        
        if 'nl' in const_dict:
            nl = const_dict['nl']
        else:
            if a_is_free:
                v = a
                a_is_free = False
            elif b_is_free:
                v = b
                b_is_free = False
            else:
                raise ValueError
            nl = v
            
        if 'sz' in const_dict:
            sz = const_dict['sz']
        else:
            if a_is_free:
                v = a
                a_is_free = False
            elif b_is_free:
                v = b
                b_is_free = False
            else:
                raise ValueError
            sz = v
        
        if 'lr' in const_dict:
            lr = const_dict['lr']
        else:
            if a_is_free:
                v = a
                a_is_free = False
            elif b_is_free:
                v = b
                b_is_free = False
            else:
                raise ValueError
            lr = v
        
        if 'bs' in const_dict:
            bs = const_dict['bs']
        else:
            if a_is_free:
                v = a
                a_is_free = False
            elif b_is_free:
                v = b
                b_is_free = False
            else:
                raise ValueError
            bs = v
            
        csv_basename += nl + 'x' + sz + 'd5_lr' + lr + '_b' + bs
        
        if not os.path.exists(top_models_dir + csv_basename + ".csv"):
            continue
            
        color = special_color.get(csv_basename, c)
        csv_data = pd.read_csv(top_models_dir + csv_basename + ".csv")
        xscale = epoch_time[(nl, sz, bs)] if in_seconds else 1
        axes.plot(csv_data.epoch * xscale, csv_data.val_loss, color=c)
        legend_labels.append(', '.join([a, b]))

    title = ''
    title += const_dict['nl'] if 'nl' in const_dict else '?'
    title += 'x'
    title += const_dict['sz'] if 'sz' in const_dict else '????'
    title += 'd5_lr'
    title += const_dict['lr'] if 'lr' in const_dict else '??'
    title += '_b'
    title += const_dict['bs'] if 'bs' in const_dict else '??'
        
    if ncol >= 3:
        fontsize = 13
    else:
        fontsize = 16
    axes.set_title(coco_prefix + title)
    xlabel = r'Time (s)' if in_seconds else r'Epoch'
    axes.set_xlabel(xlabel)
    axes.set_ylabel(r'Validation Loss')
    if xlim is not None:
        axes.set_xlim(xlim)
        
    axes.set_ylim(ylim)
#     axes.legend(legend_labels, ncol=ncol, loc='upper right', bbox_to_anchor=(0.99, 0.99), fontsize=fontsize)
    axes.legend(legend_labels, ncol=ncol, loc='upper right', bbox_to_anchor=(0.99, 0.99))

In [ ]:
nl_sz_c = [
    ("1", "1024", LIME_GREEN),
    ("1", "2048", 'g'),
    ("1", "4096", "y"),
    ("2", "1024", ORANGE),
    ("2", "2048", "r"),
    ("2", "4096", "c"),
]
fixed_vals = ['lr', 'bs']
for const_a in hyper_dict[fixed_vals[0]]:
    for const_b in hyper_dict[fixed_vals[1]]:
        const_args = {fixed_vals[0]: const_a, fixed_vals[1]: const_b}
        plot_compare(const_args, nl_sz_c, ncol=1, xlim=(0, 200))

In [ ]:
nl_lr_c = [
    ("1", "52", RUST),
    ("1", "11", "y"),
    ("2", "52", "b"),
    ("2", "11", "c"),
]
fixed_vals = ['sz', 'bs']
for const_a in hyper_dict[fixed_vals[0]]:
    for const_b in hyper_dict[fixed_vals[1]]:
        const_args = {fixed_vals[0]: const_a, fixed_vals[1]: const_b}
        plot_compare(const_args, nl_lr_c, ncol=1, xlim=(0, 200))

In [ ]:
nl_bs_c = [
    ("1", "128", "g"),
    ("1", "64", "c"),
    ("1", "32", "b"),
    ("2", "128", "m"),
    ("2", "64", "r"),
    ("2", "32", "y"),
]
fixed_vals = ['sz', 'lr']
for const_a in hyper_dict[fixed_vals[0]]:
    for const_b in hyper_dict[fixed_vals[1]]:
        const_args = {fixed_vals[0]: const_a, fixed_vals[1]: const_b}
        plot_compare(const_args, nl_bs_c, ncol=1)

In [ ]:
nl_bs_c = [
    ("1", "32", "b"),
    ("1", "64", "c"),
    ("1", "128", "g"),
    ("2", "32", "y"),
    ("2", "64", "r"),
    ("2", "128", "m"),
]
fixed_vals = ['sz', 'lr']
for const_a in hyper_dict[fixed_vals[0]]:
    for const_b in hyper_dict[fixed_vals[1]]:
        const_args = {fixed_vals[0]: const_a, fixed_vals[1]: const_b}
        plot_compare(const_args, nl_bs_c, in_seconds=True, ncol=1)

In [ ]:
sz_lr_c = [
    ("1024", "52", BROWN),
    ("2048", "52", PINK),
    ("4096", "52", 'b'),
    ("1024", "11", ORANGE),
    ("2048", "11", "r"),
    ("4096", "11", "c"),
]
fixed_vals = ['nl', 'bs']
for const_a in hyper_dict[fixed_vals[0]]:
    for const_b in hyper_dict[fixed_vals[1]]:
        const_args = {fixed_vals[0]: const_a, fixed_vals[1]: const_b}
        plot_compare(const_args, sz_lr_c, ncol=1, xlim=(0, 200))

In [ ]:
sz_bs_c = [
    ("1024", "128", SEA_BLUE),
    ("2048", "128", RUST),
    ("4096", "128", PINK),
    ("1024", "64", ORANGE),
    ("2048", "64", PURPLE),
    ("4096", "64", GREY),
    ("1024", "32", LIME_GREEN),
    ("2048", "32", BROWN),
    ("4096", "32", LIGHT_SALMON),
]
fixed_vals = ['nl', 'lr']
for const_a in hyper_dict[fixed_vals[0]]:
    for const_b in hyper_dict[fixed_vals[1]]:
        const_args = {fixed_vals[0]: const_a, fixed_vals[1]: const_b}
        plot_compare(const_args, sz_bs_c, ncol=3, ylim=(0.066, 0.08))

In [ ]:
sz_bs_c = [
    ("1024", "128", SEA_BLUE),
    ("2048", "128", RUST),
    ("4096", "128", PINK),
    ("1024", "64", ORANGE),
    ("2048", "64", PURPLE),
    ("4096", "64", GREY),
    ("1024", "32", LIME_GREEN),
    ("2048", "32", BROWN),
    ("4096", "32", LIGHT_SALMON),
]
fixed_vals = ['nl', 'lr']
for const_a in hyper_dict[fixed_vals[0]]:
    for const_b in hyper_dict[fixed_vals[1]]:
        const_args = {fixed_vals[0]: const_a, fixed_vals[1]: const_b}
        plot_compare(const_args, sz_bs_c, in_seconds=True, ncol=3, ylim=(0.066, 0.08))

In [ ]:
lr_bs_c = [
    ("11", "64", 'm'),
    ("52", "32", GREY),
    ("11", "128", 'c'),
    ("52", "64", LIGHT_SALMON),
    ("11", "256", 'k'),
    ("52", "128", 'b'),
]

fixed_vals = ['nl', 'sz']
for const_a in hyper_dict[fixed_vals[0]]:
    for const_b in hyper_dict[fixed_vals[1]]:
        const_args = {fixed_vals[0]: const_a, fixed_vals[1]: const_b}
        plot_compare(const_args, lr_bs_c, ncol=3, xlim=(0, 200))

In [ ]:
lr_bs_c = [
    ("11", "64", 'm'),
    ("11", "128", 'c'),
    ("11", "256", 'k'),
    ("52", "32", GREY),
    ("52", "64", LIGHT_SALMON),
    ("52", "128", 'b'),
]

# fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))
# legend_labels = []
# for lr, bs, c in lr_bs_c:
#     csv_file = "../data/bottleneck_top_models/coco_2017_zmuv_avg_2x4096d5_lr" + lr + "_b" + bs + ".csv"
#     csv_data = pd.read_csv(csv_file)
#     axes.plot(csv_data.epoch * epoch_time[bs], csv_data.val_loss, color=c)
#     legend_labels.append(', '.join([lr, bs]))
    
# axes.set_title("coco_2017_zmuv_avg_2x4096d5_lr??_b??", {'fontsize': 20})
# axes.set_xlabel(r'Time (s)', {'fontsize': 20})
# axes.set_ylabel(r'Validation Loss', {'fontsize': 20})
# axes.set_ylim((0.065, 0.085))
# axes.legend(legend_labels, ncol=2, loc='upper right', bbox_to_anchor=(0.98, 0.97))

fixed_vals = ['nl', 'sz']
for const_a in hyper_dict[fixed_vals[0]]:
    for const_b in hyper_dict[fixed_vals[1]]:
        const_args = {fixed_vals[0]: const_a, fixed_vals[1]: const_b}
        plot_compare(const_args, lr_bs_c, in_seconds=True)

In [ ]:
csv_basefiles = {
    "coco_2017_zmuv_avg_1x1024d5_lr11_b16": "r",
    "coco_2017_zmuv_avg_1x1024d5_lr12_b16": "c",
    "coco_2017_zmuv_avg_1x1024d5_lr13_b04": "b",
    "coco_2017_zmuv_avg_1x1024d5_lr11_b32": "g",
    "coco_2017_zmuv_avg_1x1024d5_lr12_b32": "m",
    "coco_2017_zmuv_avg_1x1024d5_lr13_b32": "y",
}

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))

for csv_basefile, color in csv_basefiles.items():
    csv_file = "../data/bottleneck_top_models/" + csv_basefile + ".csv"
    csv_data = pd.read_csv(csv_file)
    csv_data.val_loss.plot(ax=axes, color=color)

axes.set_title("coco_2017_zmuv_avg_1x1024d5_????_???", {'fontsize': 20})
axes.set_xlabel(r'Epoch', {'fontsize': 20})
axes.set_ylabel(r'Validation Loss', {'fontsize': 20})
legend_labels = ['lr11, b16', 'lr12, b16', 'lr13, b16', 'lr11, b32', 'lr12, b32', 'lr13, b32']
axes.legend(legend_labels, ncol=2, loc='upper right', bbox_to_anchor=(0.98, 0.97))

# Train Model

In [ ]:
from transfer import KerasTransfer

kt = KerasTransfer(
    dataset='coco',
    subset='2017',
    base_pooling='avg',
    pp_method='zmuv',
    hidden_size_1=4096,
    hidden_size_2=4096,
    batch_norm=False,
    dropout=5,
    learn_rate_pretrain=(1, 1),
    batch_size_pretrain=32,
    resume_pretraining=True,
    n_frozen=15,
    learn_rate=(1, 1),
    batch_size=128,
    epochs=500,
    transform_flags='hwrszf'
)

In [ ]:
kt.load_top_model()
print(kt.final_weights_file)
kt.train()

In [ ]:
csv_top_model_template = '../data/final_weights/coco*.csv'
min_losses = []
for csv_file in sorted(glob(csv_top_model_template)):
    csv_basefile = csv_file.rsplit(os.sep, maxsplit=1)[1]
    csv_data = pd.read_csv(csv_file)
    min_idx = np.argmin(csv_data.val_loss)
    n_epochs = len(csv_data.val_loss)
    min_losses.append((csv_basefile, n_epochs, n_epochs - min_idx, csv_data.val_loss[min_idx]))

for label, n_ep, from_end, min_loss in sorted(min_losses, key=lambda x: x[3]):
    print("{:8.5f} {:3} {:2} {}".format(min_loss, n_ep, from_end, label))

# Testing and Evaluation COCO

In [ ]:
from transfer import KerasTransfer
from coco import CocoGenerator
import metrics

valid_generator = CocoGenerator(split_name='valid')

In [ ]:
subset_name = '2017'

kt = KerasTransfer(
    dataset='coco',
    subset=subset_name,
    base_pooling='avg',
    pp_method='zmuv',
    hidden_size_1=4096,
    hidden_size_2=4096,
    batch_norm=False,
    learn_rate_pretrain=(1, 1),
    batch_size_pretrain=32,
    n_frozen=15,
    learn_rate=(1, 1),
    batch_size=128,
    epochs=500,
    transform_flags='hwrszf'
)

In [ ]:
# Calculate precision, recall, and F1 score on all results matching csv_top_model_template
csv_top_model_template = '../data/final_weights/coco*.csv'
to_skip = [
    'coco_2017_zmuv_avg_1x2048bn_lr12_b64_15_lr12_b64_f',
    'coco_2017_zmuv_avg_2x4096d5_lr11_b128_15_lr11_b128_hwrszf'
]
min_losses = []

for csv_file in sorted(glob(csv_top_model_template)):
    csv_basefile = csv_file.rsplit(os.sep, maxsplit=1)[1]
    basefile = csv_basefile.rsplit('.', maxsplit=1)[0]

    if basefile in to_skip:
        continue

    csv_data = pd.read_csv(csv_file)
    min_idx = np.argmin(csv_data.val_loss)
    min_loss = csv_data.val_loss[min_idx]
    min_losses.append((basefile, min_loss))

for basefile, min_loss in sorted(min_losses, key=lambda x: x[1]):

    kt = KerasTransfer(model_basefilename=basefile)
    y_pred_f32 = kt.load_predictions('valid')
    y_test_f64 = kt.load_labels('valid')
    y_test_f32 = y_test_f64.astype(np.float32)
    y_test_ui8 = y_test_f32.astype(np.uint8)
    y_test_bool = y_test_ui8.astype(np.bool_)

    indices_k3 = np.where(np.sum(y_test_bool, axis=1) == 3)[0]
    y_pred_f32 = y_pred_f32[indices_k3]
    y_test_f32 = y_test_f32[indices_k3]
    y_test_ui8 = y_test_ui8[indices_k3]
    y_test_bool = y_test_bool[indices_k3]

    y_pred_bool_k = metrics.get_k_predictions(y_test_f32, y_pred_f32)
    y_pred_bool_3 = metrics.get_k_predictions(y_test_f32, y_pred_f32, k=3)

    macro_k = metrics.get_precision_recall_fscore(y_test_bool, y_pred_bool_k, 'macro')
    micro_k = metrics.get_precision_recall_fscore(y_test_bool, y_pred_bool_k, 'micro')
    macro_3 = metrics.get_precision_recall_fscore(y_test_bool, y_pred_bool_3, 'macro')
    micro_3 = metrics.get_precision_recall_fscore(y_test_bool, y_pred_bool_3, 'micro')
    print('{0[0]:>.1%} {0[1]:>.1%} {0[2]:>.1%} | {1[2]:>.1%} | {2[0]:>.1%} {2[1]:>.1%} {2[2]:>.1%} | {3[0]:>.1%} {3[1]:>.1%} {3[2]:>.1%} | {4:>6.4f} {5}'.format(macro_k, micro_k, macro_3, micro_3, min_loss, basefile.split('avg_')[1]))

In [ ]:
kt = KerasTransfer(model_basefilename='coco_2017_zmuv_avg_2x4096d5_lr11_b256_15_lr11_b128_hwrszf')
# Augmenting a dataset takes time. Let's do it once and keep a reference to them.
# y_pred_f32_aug = kt.make_predictions('valid', augment=True)

In [ ]:
y_pred_f32 = kt.load_predictions('valid')
# y_pred_f32 = y_pred_f32_aug
print(type(y_pred_f32[1, 1]), y_pred_f32.shape)

In [ ]:
y_test_f64 = kt.load_labels('valid')
print(type(y_test_f64[1, 1]), y_test_f64.shape)

In [ ]:
# x_test_ui8 = kt.load_tensors('valid')
# print(type(x_test_ui8[1, 1, 1, 1]), x_test_ui8.shape)

In [ ]:
# print(np.sum(y_test_f64))
# temp = y_test_f64.shape
# print(temp[0] * temp[1])

y_test_f32 = y_test_f64.astype(np.float32)
print(type(y_test_f32[1, 1]), y_test_f32.shape)
y_test_ui8 = y_test_f32.astype(np.uint8)
print(type(y_test_ui8[1, 1]), y_test_ui8.shape)
y_test_bool = y_test_ui8.astype(np.bool_)
print(type(y_test_bool[1, 1]), y_test_bool.shape)

In [ ]:
# Create a specal case with perfect predictions.
y_pred_f32 = np.random.random(y_test_bool.shape)*0.1
y_pred_f32[y_test_bool] = y_pred_f32[y_test_bool]+0.9
print(y_pred_f32.shape)

In [ ]:
# Remove empty label samples
indices_k3 = np.where(np.sum(y_test_bool, axis=1) > 0)[0]
# Remove all samples that don't have exactly 3 labels.
# indices_k3 = np.where(np.sum(y_test_bool, axis=1) == 3)[0]
y_pred_f32 = y_pred_f32[indices_k3]
y_test_f32 = y_test_f32[indices_k3]
y_test_ui8 = y_test_ui8[indices_k3]
y_test_bool = y_test_bool[indices_k3]
print(type(y_pred_f32[1, 1]), y_pred_f32.shape)
print(type(y_test_f32[1, 1]), y_test_f32.shape)
print(type(y_test_ui8[1, 1]), y_test_ui8.shape)
print(type(y_test_bool[1, 1]), y_test_bool.shape)

In [ ]:
# Apply some label decision methods
y_pred_bool_k = metrics.get_k_predictions(y_test_f32, y_pred_f32)
y_pred_bool_3 = metrics.get_k_predictions(y_test_f32, y_pred_f32, k=3)
y_pred_bool_tau = metrics.get_tau_predictions(y_test_f32, y_pred_f32)
y_pred_bool_taus = metrics.get_tau_predictions(y_test_f32, y_pred_f32, tau=-1)
print(type(y_pred_bool_k[1, 1]), y_pred_bool_k.shape)
print(type(y_pred_bool_3[1, 1]), y_pred_bool_3.shape)
print(type(y_pred_bool_tau[1, 1]), y_pred_bool_tau.shape)
print(type(y_pred_bool_taus[1, 1]), y_pred_bool_taus.shape)

In [ ]:
np.sum(y_pred_bool_k, axis=0)
np.sum(y_pred_bool_taus, axis=0)

In [ ]:
metrics.print_precision_recall_fscore(y_test_bool, y_pred_bool_k)
print('')
metrics.print_precision_recall_fscore(y_test_bool, y_pred_bool_3)
print('')
metrics.print_precision_recall_fscore(y_test_bool, y_pred_bool_tau)
print('')
metrics.print_precision_recall_fscore(y_test_bool, y_pred_bool_taus)

In [ ]:
# Let's see where we performed the worst.

EPS = np.finfo(float).eps

a = np.sum(y_pred_bool_k & y_test_bool, axis=1)
b = np.sum(y_pred_bool_k, axis=1) + EPS
c = np.sum(y_test_bool, axis=1) + EPS
p_samp_k = a / b
r_samp_k = a / c
f1_samp_k = a / (b + c)

bad_p_indices_k = np.argsort(p_samp_k)
bad_r_indices_k = np.argsort(r_samp_k)
bad_f1_indices_k = np.argsort(f1_samp_k)

print(p_samp_k.shape, bad_p_indices_k[:5])
print(r_samp_k.shape, bad_r_indices_k[:5])
print(f1_samp_k.shape, bad_f1_indices_k[:5])

In [ ]:
idx = 480
print(y_test_bool[idx].astype(np.uint8))
print(y_pred_f32[idx])
print(y_pred_bool_k[idx].astype(np.uint8))
print(y_pred_bool_tau[idx].astype(np.uint8))

In [ ]:
biggest_test_example_index = np.argmax(np.sum(y_test_f32, axis=1))

In [ ]:
biggest_test_example_index = 480

In [ ]:
# img = x_test_ui8[biggest_test_example_index]
img = valid_generator.load_image(biggest_test_example_index)[0]
plt.imshow(img)
plt.xticks([])
plt.yticks([])
plt.show()

print(biggest_test_example_index)

ranks = np.argsort(y_pred_f32[biggest_test_example_index])[::-1]
biggest_test_example_dict = {
    'y_test_f32': y_test_f32[biggest_test_example_index][ranks],
    'y_pred_f32': y_pred_f32[biggest_test_example_index][ranks],
    'y_pred_bool_k': y_pred_bool_k[biggest_test_example_index][ranks],
    'y_pred_bool_tau': y_pred_bool_tau[biggest_test_example_index][ranks],
}

biggest_test_example_df = pd.DataFrame(biggest_test_example_dict, index=valid_generator.label_names[ranks])
biggest_test_example_df.style

In [ ]:
sz = 10
si = 0
for idx in bad_f1_indices_k:
# for idx in np.random.randint(len(y_test_ui8), size=5):
    if np.sum(y_test_ui8[idx]) != 3:
        continue
    
    si += 1    
    img = valid_generator.load_image(idx)[0]
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    plt.show()
    
    test_indices = np.where(y_test_ui8[idx])[0]
    pred_indices = np.argsort(y_pred_f32[idx])[::-1]
    
    ranks = []
    y_test = []
    y_pred = []
    labels = []
    for i, pi in enumerate(pred_indices):
        if i < len(test_indices) or pi in test_indices:
            ranks.append(i+1)
            y_test.append(y_test_ui8[idx][pi])
            y_pred.append(y_pred_f32[idx][pi])
            labels.append(valid_generator.label_names[pi])
#             print(i+1, y_test_ui8[idx][pi], y_pred_f32[idx][pi], valid_generator.label_names[pi])
        
    test_example_dict = {
        'y_test': y_test,
        'y_pred': y_pred,
        'label': labels
    }
    test_example_df = pd.DataFrame(test_example_dict, index=ranks)
    print(idx)
    print(test_example_df.to_latex())
        
    if si == sz:
        break
#     print(test_indices)
#     print(y_test_ui8[idx][test_indices])
#     print(valid_generator.label_names[test_indices])

In [ ]:
test_sums_0 = np.sum(y_test_ui8, axis=0)
test_sums_1 = np.sum(y_test_ui8, axis=1)

idx = np.argmin(test_sums_1)
img = valid_generator.load_image(idx)[0]
plt.imshow(img)
plt.show()
print(y_test_ui8.shape, idx, img.shape, img.min(), img.max())
print(valid_generator.label_names[np.where(y_test_ui8[idx])[0]])

In [ ]:
def rank_sort(y_test_bool, y_pred_bool, y_pred_float):
    y_test_bool_sorted = np.empty(y_test_bool.shape, dtype=bool)
    y_pred_bool_sorted = np.empty(y_pred_bool.shape, dtype=bool)
    for i, preds in enumerate(y_pred_float):
        sort_indices = np.argsort(preds)[::-1]
        y_test_bool_sorted[i, :] = y_test_bool[i, sort_indices]
        y_pred_bool_sorted[i, :] = y_pred_bool[i, sort_indices]
    return y_test_bool_sorted, y_pred_bool_sorted

In [ ]:
y_test_bool_sorted1, y_pred_k_bool_sorted1 = rank_sort(y_test_bool, y_pred_bool_k, y_pred_f32)
y_test_bool_sorted1, y_pred_tau_bool_sorted1 = rank_sort(y_test_bool, y_pred_bool_tau, y_pred_f32)

In [ ]:
print(average_precision_score(y_test_f32, y_pred_f32, average='micro'), 'micro')
print(average_precision_score(y_test_f32, y_pred_f32, average='macro'), 'macro')
print(label_ranking_average_precision_score(y_test_f32, y_pred_f32))
print('prf k')
print(average_precision_score(y_test_bool, y_pred_bool_k, average='micro'), 'micro')
print(average_precision_score(y_test_bool, y_pred_bool_k, average='macro'), 'macro')
print(average_precision_score(y_test_bool_sorted1, y_pred_k_bool_sorted1, average='micro'))
print(average_precision_score(y_test_bool_sorted1, y_pred_k_bool_sorted1, average='macro'))
print(precision_recall_fscore_support(y_test_bool, y_pred_bool_k, average='micro'), 'micro')
print(precision_recall_fscore_support(y_test_bool, y_pred_bool_k, average='macro'), 'macro')
print('prf tau')
print(average_precision_score(y_test_bool, y_pred_bool_tau, average='micro'), 'micro')
print(average_precision_score(y_test_bool, y_pred_bool_tau, average='macro'), 'macro')
print(average_precision_score(y_test_bool_sorted1, y_pred_tau_bool_sorted1, average='micro'))
print(average_precision_score(y_test_bool_sorted1, y_pred_tau_bool_sorted1, average='macro'))
print(precision_recall_fscore_support(y_test_bool, y_pred_bool_tau, average='micro'), 'micro')
print(precision_recall_fscore_support(y_test_bool, y_pred_bool_tau, average='macro'), 'macro')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_bool, y_pred_bool_tau, target_names=valid_generator.label_names, digits=4))

In [ ]:
from operator import itemgetter
from collections import OrderedDict
from sklearn.metrics import precision_recall_fscore_support

precision, recall, fscore, support = precision_recall_fscore_support(y_test_f32, y_pred_bool_tau)
prf1_dict = OrderedDict({
    'precision': precision,
    'recall': recall,
    'fscore': fscore,
    'support': support
})

# ranks = np.argsort(y_pred_bool_tau_123[biggest_test_example_index])[::-1]
# ranks = sorted(prf1_dict, key=itemgetter('support'), reverse=True)
prf1_df = pd.DataFrame(prf1_dict, index=valid_generator.label_names)
prf1_df.sort_values(by='support', ascending=False)
# prf1_df.style